# Lab 03: CuPy & N-Body
Links:
- https://github.com/jacobtomlinson/gpu-python-tutorial
- https://docs.cupy.dev/en/stable/user_guide/index.html
- https://github.com/pleiszenburg/gravitation
- https://developer.nvidia.com/gpugems/gpugems3/part-v-physics-simulation/chapter-31-fast-n-body-simulation-cuda
- https://medium.com/swlh/create-your-own-n-body-simulation-with-python-f417234885e9

Slides:
- https://docs.google.com/presentation/d/17HOQBO87Z_kTTnaO2Tl1O-Ps1IAsr7sZt72DqylgyD4/edit?usp=sharing

## Install CuPy

In [ ]:
! pip install cupy-cuda11x --user

## Import libraries

In [ ]:
import cupy as cp
import numpy as np

## NumPy example

We'll revisit the NumPy example of calculating the invariant mass for a million relativisitic particles.

### Create CPU arrays

First we create the arrays *on the CPU*.

In [ ]:
energy = np.random.normal(100, 10, 1000000)
px = np.random.normal(0, 10, 1000000)
py = np.random.normal(0, 10, 1000000)
pz = np.random.normal(0, 10, 1000000)

### Benchmark NumPy

We can use the `%%timeit` magic command to test the time of this cell. 

From the last lab, we saw this wasn't as optimal as creating a special *kernel* that could be *just-in-time compiled* with Numba, but just as a simple benchmark.

In [ ]:
%%timeit

mass = np.sqrt(energy**2 - px**2 - py**2 - pz**2)

## CuPy example

### Create GPU arrays

Now, we can create the arrays *on the GPU*.

In [ ]:
energy_gpu = cp.random.normal(100, 10, 1000000)
px_gpu = cp.random.normal(0, 10, 1000000)
py_gpu = cp.random.normal(0, 10, 1000000)
pz_gpu = cp.random.normal(0, 10, 1000000)

In [ ]:
from cupyx.profiler import benchmark

def cp_mass(energy, px, py, pz):
    return cp.sqrt(energy**2 - px**2 - py**2 - pz**2)

In [ ]:
benchmark(cp_mass, (energy_gpu, px_gpu, py_gpu, pz_gpu), n_repeat=100)

Note, it's not a huge speedup over vanilla NumPy. Why is that?

## CuPy Kernel example

We can remove the pure Python overhead in betewen calcualtions, by creating a custom "elementwise" kernel that will run the entire calculation on the GPU.

In [ ]:
cp_mass_kernel = cp.ElementwiseKernel(
   'float64 e, float64 px, float64 py, float64 pz',
   'float64 m',
   'm = sqrt(e * e - px * px - py * py - pz * pz)',
   'cp_mass_kernel')

In [ ]:
benchmark(cp_mass_kernel, (energy_gpu, px_gpu, py_gpu, pz_gpu), n_repeat=100)

It's much faster!

## N-Body

### Loop-based

In [ ]:
def getAcc( pos, mass, G, softening ):
    """
    Calculate the acceleration on each particle due to Newton's Law 
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """
    
    N = pos.shape[0];
    a = np.zeros((N,3));
    
    for i in range(N):
        for j in range(N):
            dx = pos[j,0] - pos[i,0]
            dy = pos[j,1] - pos[i,1]
            dz = pos[j,2] - pos[i,2]
            inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)**(-1.5)
            a[i,0] +=  G * (dx * inv_r3) * mass[j]
            a[i,1] +=  G * (dy * inv_r3) * mass[j]
            a[i,2] +=  G * (dz * inv_r3) * mass[j]

    return a

### Vectorized

In [ ]:
def getAcc( pos, mass, G, softening ):
    """
    Calculate the acceleration on each particle due to Newton's Law
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """
    # positions r = [x,y,z] for all particles
    x = pos[:,0:1]
    y = pos[:,1:2]
    z = pos[:,2:3]
    
    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z
    
    # matrix that stores 1/r^3 for all particle pairwise particle separations 
    inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)**(-1.5)
    
    ax = G * (dx * inv_r3) @ mass
    ay = G * (dy * inv_r3) @ mass
    az = G * (dz * inv_r3) @ mass
    
    # pack together the acceleration components
    a = np.hstack((ax,ay,az))

    return a

In [ ]:
def getEnergy( pos, vel, mass, G ):
    """
    Get kinetic energy (KE) and potential energy (PE) of simulation
    pos is N x 3 matrix of positions
    vel is N x 3 matrix of velocities
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    KE is the kinetic energy of the system
    PE is the potential energy of the system
    """
    # Kinetic Energy:
    KE = 0.5 * np.sum(np.sum( mass * vel**2 ))

    # Potential Energy:

    # positions r = [x,y,z] for all particles
    x = pos[:,0:1]
    y = pos[:,1:2]
    z = pos[:,2:3]

    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z

    # matrix that stores 1/r for all particle pairwise particle separations 
    inv_r = np.sqrt(dx**2 + dy**2 + dz**2)
    inv_r[inv_r>0] = 1.0/inv_r[inv_r>0]

    # sum over upper triangle, to count each interaction only once
    PE = G * np.sum(np.sum(np.triu(-(mass*mass.T)*inv_r,1)))
    
    return KE, PE

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

# Simulation parameters
N         = 100    # Number of particles
t         = 0      # current time of the simulation
tEnd      = 10.0   # time at which simulation ends
dt        = 0.01   # timestep
softening = 0.1    # softening length
G         = 1.0    # Newton's Gravitational Constant
plotRealTime = False # switch on for plotting as the simulation goes along

# Generate Initial Conditions
np.random.seed(17)            # set the random number generator seed

mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
pos  = np.random.randn(N,3)   # randomly selected positions and velocities
vel  = np.random.randn(N,3)

# Convert to Center-of-Mass frame
vel -= np.mean(mass * vel,0) / np.mean(mass)

# calculate initial gravitational accelerations
acc = getAcc( pos, mass, G, softening )

# calculate initial energy of system
KE, PE  = getEnergy( pos, vel, mass, G )

# number of timesteps
Nt = int(np.ceil(tEnd/dt))

# save energies, particle orbits for plotting trails
pos_save = np.zeros((N,3,Nt+1))
pos_save[:,:,0] = pos
KE_save = np.zeros(Nt+1)
KE_save[0] = KE
PE_save = np.zeros(Nt+1)
PE_save[0] = PE
t_all = np.arange(Nt+1)*dt

    
# Simulation Main Loop
for i in range(Nt):
    # (1/2) kick
    vel += acc * dt/2.0

    # drift
    pos += vel * dt

    # update accelerations
    acc = getAcc( pos, mass, G, softening )

    # (1/2) kick
    vel += acc * dt/2.0

    # update time
    t += dt

    # get energy of system
    KE, PE  = getEnergy( pos, vel, mass, G )

    # save energies, positions for plotting trail
    pos_save[:,:,i+1] = pos
    KE_save[i+1] = KE
    PE_save[i+1] = PE

In [ ]:
for i in range(0, Nt, 10):
    plt.sca(ax1)
    plt.cla()
    xx = pos_save[:,0,max(i-50,0):i+1]
    yy = pos_save[:,1,max(i-50,0):i+1]
    plt.scatter(xx,yy,s=1,color=[.7,.7,1])
    plt.scatter(pos[:,0],pos[:,1],s=10,color='blue')
    ax1.set(xlim=(-2, 2), ylim=(-2, 2))
    ax1.set_aspect('equal', 'box')
    ax1.set_xticks([-2,-1,0,1,2])
    ax1.set_yticks([-2,-1,0,1,2])

    plt.sca(ax2)
    plt.cla()
    plt.scatter(t_all,KE_save,color='red',s=1,label='KE' if i == Nt-1 else "")
    plt.scatter(t_all,PE_save,color='blue',s=1,label='PE' if i == Nt-1 else "")
    plt.scatter(t_all,KE_save+PE_save,color='black',s=1,label='Etot' if i == Nt-1 else "")
    ax2.set(xlim=(0, tEnd), ylim=(-300, 300))
    ax2.set_aspect(0.007)

    # add labels/legend
    plt.sca(ax2)
    plt.xlabel('time')
    plt.ylabel('energy')
    ax2.legend(loc='upper right')

    # Save figure
    plt.savefig(f'nbody_{i}.png',dpi=240)

In [ ]:
! 